## Testes de valência para o projeto final de IA369Y 2 Semestre 2018

Passos para tratar os dados com valência, testar e escolher um classificador para utilizar no projeto final de IA369Y.

1) Remover espaços duplos, quebras de linha, números e links do dataset e das frases a serem testadas.

2) Remover stopwords e aplicar o stemmer.

3) Treinar os classificadores.

4) Realizar as predições com os classificadores.

5) Avaliar as medidas obtidas com os classificadores.

In [49]:
import sys
sys.path.append('../..')

import csv
import codecs
import copy
import re
from random import shuffle

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from nltk import word_tokenize
import gensim

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer, MinMaxScaler
from sklearn.decomposition import TruncatedSVD, LatentDirichletAllocation
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB, GaussianNB, ComplementNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score, confusion_matrix

from utils import tokenizer, load_six_emotions, load_3_emotions

%matplotlib inline

np.warnings.filterwarnings('ignore')
np.random.seed(12345)

def highlight_max(data, color='green'):
    attr = f'background-color: {color}; color: white; font-weight: bold;'
    #remove % and cast to float
    data = data.replace('%','', regex=True).astype(float)
    if data.ndim == 1:  # Series from .apply(axis=0) or axis=1
        is_max = data == data.max()
        return [attr if v else '' for v in is_max]
    else:  # from .apply(axis=None)
        is_max = data == data.max().max()
        return pd.DataFrame(np.where(is_max, attr, ''),
                            index=data.index, columns=data.columns)

classf = {
    'MultinomialNB': 0,
    'ComplementNB': 0,
    'LogisticRegression': 0,
    'RandomForestClassifier': 0,
    'KNeighborsClassifier': 0,
    'MLPClassifier': 0,
    'LinearSVC': 0,
    'SVC': 0
}

matriz_resultados = {
    'all': {
        'tfidf': copy.deepcopy(classf),
        'tfidf+lsa': copy.deepcopy(classf),
        'tfidf+lda': copy.deepcopy(classf),
        'count': copy.deepcopy(classf),
        'count+lsa': copy.deepcopy(classf),
        'count+lda': copy.deepcopy(classf),
        'tfidf+count+w2c': copy.deepcopy(classf),
    },
    'tweets': {
        'tfidf': copy.deepcopy(classf),
        'tfidf+lsa': copy.deepcopy(classf),
        'tfidf+lda': copy.deepcopy(classf),
        'count': copy.deepcopy(classf),
        'count+lsa': copy.deepcopy(classf),
        'count+lda': copy.deepcopy(classf),
        'tfidf+count+w2c': copy.deepcopy(classf),       
    },
    'titulo_noticias': {
        'tfidf': copy.deepcopy(classf),
        'tfidf+lsa': copy.deepcopy(classf),
        'tfidf+lda': copy.deepcopy(classf),
        'count': copy.deepcopy(classf),
        'count+lsa': copy.deepcopy(classf),
        'count+lda': copy.deepcopy(classf),
        'tfidf+count+w2c': copy.deepcopy(classf),
    }
}

## Datasets

Para validar, serão utilizados dois datasets.

Os dos datasets foram obtidos do site minerando dados.

O primeiro deles tem tweets de política de Minas Gerais com rótulos de valência: positivo, negativo e neutro. Foi feito um tratamento para eliminar tweets repetidos e dessa forma sobraram 3016 tweets.

O segundo contém 2123 títulos de notícias com rótulos de valência: positivo, negativo e neutro.

In [50]:
path = '/home/rdenadai/vagrant/python-dev/sentiment-analysis-2018-president-election/dataset/tweets/'

#Carregando os datasets
def carregar(filename):
    frases = []
    with open(filename, 'r', encoding='utf-8') as h:
        reader = csv.reader(h, delimiter='|')
        for row in reader:
            frase = tokenizer(row[0]).strip()
            valencia = row[1].upper()
            if len(frase) > 5:
                frases.append((valencia, frase))
    return frases

In [51]:
#Carrega os datasets em separado
tweets_mg = carregar(f'{path}tweets_mg_tratados.csv')
shuffle(tweets_mg)
titulo_noticias = carregar(f'{path}titulo_noticias.txt')
shuffle(titulo_noticias)

frases = tweets_mg + titulo_noticias
shuffle(frases)

print(frases[:5])
print('-' * 20)
print(tweets_mg[:5])
print('-' * 20)
print(titulo_noticias[:5])

[('NEGATIVO', 'sud min ger abrir seleçõ professor'), ('NEUTRO', 'odecarvalh certez govern obam tent imped poss trump histór hackers russ prepar'), ('NEGATIVO', 'bovesp fech qued mais pux val notíc merc'), ('NEGATIVO', 'escal tensã ucrân pressã val petrobr faz ibovesp cair'), ('NEGATIVO', 'nã recurs gast helicópter')]
--------------------
[('NEUTRO', 'gonzal tan sol con cambi estar anim total'), ('POSITIVO', 'igornasc igornasc menor det pedr crack outr drog ipating'), ('NEGATIVO', 'empresári relat moment terror sob mir armar ger estar min'), ('POSITIVO', 'possível reintegr'), ('NEGATIVO', 'advog visit cunh repass impressã subchef cas civil politic estar min')]
--------------------
[('NEUTRO', 'copel ped suspensã reajust tarif energ paran onlin paranaens'), ('NEGATIVO', 'blog esmael crític rich suspend tarifac energ copel'), ('NEGATIVO', 'dilm santand inadmiss lament brasil'), ('NEGATIVO', 'petrobr nã inform conselheir acord govern notic uol econom'), ('NEUTRO', 'tim part timp esclarec')

In [52]:
#all_datasets
afrases = []
avalencias = []
for valencia, frase in frases:
    afrases.append(frase)
    avalencias.append(valencia)
    
print(afrases[:5])
print(avalencias[:5])
print('-' * 20)

#tweets_mg
atweets_mg = []
aval_tweets_mg = []
for valencia, frase in tweets_mg:
    atweets_mg.append(frase)
    aval_tweets_mg.append(valencia)

print(atweets_mg[:5])
print(aval_tweets_mg[:5])
print('-' * 20)

#titulo_noticias
atitulo_noticias = []
aval_titulo_noticias = []
for valencia, frase in titulo_noticias:
    atitulo_noticias.append(frase)
    aval_titulo_noticias.append(valencia)

print(atitulo_noticias[:5])
print(aval_titulo_noticias[:5])

['sud min ger abrir seleçõ professor', 'odecarvalh certez govern obam tent imped poss trump histór hackers russ prepar', 'bovesp fech qued mais pux val notíc merc', 'escal tensã ucrân pressã val petrobr faz ibovesp cair', 'nã recurs gast helicópter']
['NEGATIVO', 'NEUTRO', 'NEGATIVO', 'NEGATIVO', 'NEGATIVO']
--------------------
['gonzal tan sol con cambi estar anim total', 'igornasc igornasc menor det pedr crack outr drog ipating', 'empresári relat moment terror sob mir armar ger estar min', 'possível reintegr', 'advog visit cunh repass impressã subchef cas civil politic estar min']
['NEUTRO', 'POSITIVO', 'NEGATIVO', 'POSITIVO', 'NEGATIVO']
--------------------
['copel ped suspensã reajust tarif energ paran onlin paranaens', 'blog esmael crític rich suspend tarifac energ copel', 'dilm santand inadmiss lament brasil', 'petrobr nã inform conselheir acord govern notic uol econom', 'tim part timp esclarec']
['NEUTRO', 'NEGATIVO', 'NEGATIVO', 'NEGATIVO', 'NEUTRO']


In [53]:
def run_ml_model(model, X_train, y_train, X_test, y_test):
    model.fit(X_train, y_train)
    accuracy = np.round(model.score(X_test, y_test) * 100, 2)
    print(f'Modelo   : {model.__class__.__name__}')
    print(f'Acurácia : {accuracy}%')
    print('-' * 20)
    return accuracy

def split_data(X, y):
    test_size = .33
    random_state = 0
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)
    return {
        'X_train': X_train,
        'y_train': y_train,
        'X_test': X_test,
        'y_test': y_test
    }

## Classificadores

In [54]:
classifiers = (
    MultinomialNB(),
    ComplementNB(),
    LogisticRegression(multi_class='auto', solver='lbfgs'),
    RandomForestClassifier(n_estimators=50, min_samples_split=5, random_state=0),
    KNeighborsClassifier(n_neighbors=8, algorithm='auto'),
    MLPClassifier(hidden_layer_sizes=(100, 25), max_iter=500, random_state=0),
    LinearSVC(max_iter=500),
    SVC(gamma='auto', max_iter=500),
)

## TF-IDF

In [55]:
vec_tfidf = TfidfVectorizer(ngram_range=(1, 2))
X_tfidf = vec_tfidf.fit_transform(afrases)

vec_tfidf_tmg = TfidfVectorizer(ngram_range=(1, 2))
X_tfidf_tmg = vec_tfidf_tmg.fit_transform(atweets_mg)

vec_tfidf_tn = TfidfVectorizer(ngram_range=(1, 2))
X_tfidf_tn = vec_tfidf_tn.fit_transform(atitulo_noticias)

In [56]:
print("\nall_datasets")
for classifier in classifiers:
    try:
        acc = run_ml_model(classifier, **split_data(X_tfidf, avalencias))
        matriz_resultados['all']['tfidf'][classifier.__class__.__name__] = acc
    except:
        pass

print("\ntweets_mg")
for classifier in classifiers:
    try:
        acc = run_ml_model(classifier, **split_data(X_tfidf_tmg, aval_tweets_mg))
        matriz_resultados['tweets']['tfidf'][classifier.__class__.__name__] = acc
    except:
        pass

print("\ntitulo_noticias")
for classifier in classifiers:
    try:
        acc = run_ml_model(classifier, **split_data(X_tfidf_tn, aval_titulo_noticias))
        matriz_resultados['titulo_noticias']['tfidf'][classifier.__class__.__name__] = acc
    except:
        pass      


all_datasets
Modelo   : MultinomialNB
Acurácia : 61.63%
--------------------
Modelo   : ComplementNB
Acurácia : 62.17%
--------------------
Modelo   : LogisticRegression
Acurácia : 66.43%
--------------------
Modelo   : RandomForestClassifier
Acurácia : 63.88%
--------------------
Modelo   : KNeighborsClassifier
Acurácia : 61.16%
--------------------
Modelo   : MLPClassifier
Acurácia : 64.18%
--------------------
Modelo   : LinearSVC
Acurácia : 67.32%
--------------------
Modelo   : SVC
Acurácia : 58.26%
--------------------

tweets_mg
Modelo   : MultinomialNB
Acurácia : 63.5%
--------------------
Modelo   : ComplementNB
Acurácia : 59.35%
--------------------
Modelo   : LogisticRegression
Acurácia : 65.62%
--------------------
Modelo   : RandomForestClassifier
Acurácia : 64.91%
--------------------
Modelo   : KNeighborsClassifier
Acurácia : 60.67%
--------------------
Modelo   : MLPClassifier
Acurácia : 60.57%
--------------------
Modelo   : LinearSVC
Acurácia : 62.49%
---------------

## LSA (usando TF-IDF)

In [57]:
#all_datasets
svd = TruncatedSVD(n_components=70, n_iter=50, random_state=0)
normalizer = MinMaxScaler(copy=False)
lsa = make_pipeline(svd, normalizer)
X_svd = lsa.fit_transform(X_tfidf)

print("\nall_datasets")
for classifier in classifiers:
    try:
        acc = run_ml_model(classifier, **split_data(X_svd, avalencias))
        matriz_resultados['all']['tfidf+lsa'][classifier.__class__.__name__] = acc
    except Exception as e:
        print(e)

#tweets_mg
X_svd = lsa.fit_transform(X_tfidf_tmg)

print("\ntweets_mg")
for classifier in classifiers:
    try:
        acc = run_ml_model(classifier, **split_data(X_svd, aval_tweets_mg))
        matriz_resultados['tweets']['tfidf+lsa'][classifier.__class__.__name__] = acc
    except:
        pass


#titulo_noticias
X_svd = lsa.fit_transform(X_tfidf_tn)

print("\ntitulo_noticias")
for classifier in classifiers:
    try:
        acc = run_ml_model(classifier, **split_data(X_svd, aval_titulo_noticias))
        matriz_resultados['titulo_noticias']['tfidf+lsa'][classifier.__class__.__name__] = acc
    except:
        pass


all_datasets
Modelo   : MultinomialNB
Acurácia : 45.41%
--------------------
Modelo   : ComplementNB
Acurácia : 53.52%
--------------------
Modelo   : LogisticRegression
Acurácia : 61.99%
--------------------
Modelo   : RandomForestClassifier
Acurácia : 61.28%
--------------------
Modelo   : KNeighborsClassifier
Acurácia : 57.9%
--------------------
Modelo   : MLPClassifier
Acurácia : 62.7%
--------------------
Modelo   : LinearSVC
Acurácia : 62.52%
--------------------
Modelo   : SVC
Acurácia : 47.72%
--------------------

tweets_mg
Modelo   : MultinomialNB
Acurácia : 44.29%
--------------------
Modelo   : ComplementNB
Acurácia : 56.62%
--------------------
Modelo   : LogisticRegression
Acurácia : 66.03%
--------------------
Modelo   : RandomForestClassifier
Acurácia : 62.08%
--------------------
Modelo   : KNeighborsClassifier
Acurácia : 60.87%
--------------------
Modelo   : MLPClassifier
Acurácia : 66.53%
--------------------
Modelo   : LinearSVC
Acurácia : 65.42%
----------------

## LDA (usando TF-IDF)

In [58]:
#all_datasets
lda = LatentDirichletAllocation(n_components=70, max_iter=50, random_state=0, n_jobs=5)
normalizer = MinMaxScaler(copy=False)
lda = make_pipeline(lda, normalizer)
X_lda = lda.fit_transform(X_tfidf)

print("\nall_datasets")
for classifier in classifiers:
    try:
        acc = run_ml_model(classifier, **split_data(X_lda, avalencias))
        matriz_resultados['all']['tfidf+lda'][classifier.__class__.__name__] = acc
    except:
        pass


#tweets_mg
X_lda = lda.fit_transform(X_tfidf_tmg)

print("\ntweets_mg")
for classifier in classifiers:
    try:
        acc = run_ml_model(classifier, **split_data(X_lda, aval_tweets_mg))
        matriz_resultados['tweets']['tfidf+lda'][classifier.__class__.__name__] = acc
    except:
        pass


#titulo_noticias
X_lda = lda.fit_transform(X_tfidf_tn)

print("\ntitulo_noticias")
for classifier in classifiers:
    try:
        acc = run_ml_model(classifier, **split_data(X_lda, aval_titulo_noticias))
        matriz_resultados['titulo_noticias']['tfidf+lda'][classifier.__class__.__name__] = acc
    except:
        pass      


all_datasets
Modelo   : MultinomialNB
Acurácia : 46.65%
--------------------
Modelo   : ComplementNB
Acurácia : 43.04%
--------------------
Modelo   : LogisticRegression
Acurácia : 46.36%
--------------------
Modelo   : RandomForestClassifier
Acurácia : 45.71%
--------------------
Modelo   : KNeighborsClassifier
Acurácia : 43.1%
--------------------
Modelo   : MLPClassifier
Acurácia : 47.13%
--------------------
Modelo   : LinearSVC
Acurácia : 46.54%
--------------------
Modelo   : SVC
Acurácia : 46.77%
--------------------

tweets_mg
Modelo   : MultinomialNB
Acurácia : 52.58%
--------------------
Modelo   : ComplementNB
Acurácia : 48.33%
--------------------
Modelo   : LogisticRegression
Acurácia : 53.29%
--------------------
Modelo   : RandomForestClassifier
Acurácia : 50.96%
--------------------
Modelo   : KNeighborsClassifier
Acurácia : 46.51%
--------------------
Modelo   : MLPClassifier
Acurácia : 51.47%
--------------------
Modelo   : LinearSVC
Acurácia : 53.29%
---------------

## Count

In [59]:
#all_datasets
vec_count = CountVectorizer(ngram_range=(1, 2))
X_count = vec_count.fit_transform(afrases)

print("\nall_datasets")
for classifier in classifiers:
    try:
        acc = run_ml_model(classifier, **split_data(X_count, avalencias))
        matriz_resultados['all']['count'][classifier.__class__.__name__] = acc
    except:
        pass

      
#tweets_mg
vec_count = CountVectorizer(ngram_range=(1, 2))
X_count_tmg = vec_count.fit_transform(atweets_mg)

print("\ntweets_mg")
for classifier in classifiers:
    try:
        acc = run_ml_model(classifier, **split_data(X_count_tmg, aval_tweets_mg))
        matriz_resultados['tweets']['count'][classifier.__class__.__name__] = acc
    except:
        pass
      
      
#titulo_noticias
vec_count = CountVectorizer(ngram_range=(1, 2))
X_count_tn = vec_count.fit_transform(atitulo_noticias)

print("\ntitulo_noticias")
for classifier in classifiers:
    try:
        acc = run_ml_model(classifier, **split_data(X_count_tn, aval_titulo_noticias))
        matriz_resultados['titulo_noticias']['count'][classifier.__class__.__name__] = acc
    except:
        pass      


all_datasets
Modelo   : MultinomialNB
Acurácia : 65.19%
--------------------
Modelo   : ComplementNB
Acurácia : 59.5%
--------------------
Modelo   : LogisticRegression
Acurácia : 65.96%
--------------------
Modelo   : RandomForestClassifier
Acurácia : 65.48%
--------------------
Modelo   : KNeighborsClassifier
Acurácia : 40.2%
--------------------
Modelo   : MLPClassifier
Acurácia : 63.11%
--------------------
Modelo   : LinearSVC
Acurácia : 65.01%
--------------------
Modelo   : SVC
Acurácia : 49.44%
--------------------

tweets_mg
Modelo   : MultinomialNB
Acurácia : 60.57%
--------------------
Modelo   : ComplementNB
Acurácia : 59.66%
--------------------
Modelo   : LogisticRegression
Acurácia : 61.07%
--------------------
Modelo   : RandomForestClassifier
Acurácia : 63.8%
--------------------
Modelo   : KNeighborsClassifier
Acurácia : 51.37%
--------------------
Modelo   : MLPClassifier
Acurácia : 60.47%
--------------------
Modelo   : LinearSVC
Acurácia : 60.57%
-----------------

## LSA (usando Count)

In [60]:
#all_datasets
svd = TruncatedSVD(n_components=70, n_iter=50, random_state=0)
normalizer = MinMaxScaler(copy=False)
lda = make_pipeline(svd, normalizer)
X_svd = lda.fit_transform(X_count)

print("\nall_datasets")
for classifier in classifiers:
    try:
        acc = run_ml_model(classifier, **split_data(X_svd, avalencias))
        matriz_resultados['all']['count+lsa'][classifier.__class__.__name__] = acc
    except:
        pass
      

#tweets_mg
X_svd = lda.fit_transform(X_count_tmg)

print("\ntweets_mg")
for classifier in classifiers:
    try:
        acc = run_ml_model(classifier, **split_data(X_svd, aval_tweets_mg))
        matriz_resultados['tweets']['count+lsa'][classifier.__class__.__name__] = acc
    except:
        pass
      
      
#titulos_noticias
X_svd = lda.fit_transform(X_count_tn)

print("\ntitulos_noticias")
for classifier in classifiers:
    try:
        acc = run_ml_model(classifier, **split_data(X_svd, aval_titulo_noticias))
        matriz_resultados['titulo_noticias']['count+lsa'][classifier.__class__.__name__] = acc
    except:
        pass      


all_datasets
Modelo   : MultinomialNB
Acurácia : 46.48%
--------------------
Modelo   : ComplementNB
Acurácia : 54.17%
--------------------
Modelo   : LogisticRegression
Acurácia : 61.1%
--------------------
Modelo   : RandomForestClassifier
Acurácia : 59.86%
--------------------
Modelo   : KNeighborsClassifier
Acurácia : 57.79%
--------------------
Modelo   : MLPClassifier
Acurácia : 61.81%
--------------------
Modelo   : LinearSVC
Acurácia : 61.93%
--------------------
Modelo   : SVC
Acurácia : 46.71%
--------------------

tweets_mg
Modelo   : MultinomialNB
Acurácia : 50.46%
--------------------
Modelo   : ComplementNB
Acurácia : 58.65%
--------------------
Modelo   : LogisticRegression
Acurácia : 64.41%
--------------------
Modelo   : RandomForestClassifier
Acurácia : 62.39%
--------------------
Modelo   : KNeighborsClassifier
Acurácia : 61.68%
--------------------
Modelo   : MLPClassifier
Acurácia : 66.13%
--------------------
Modelo   : LinearSVC
Acurácia : 65.22%
---------------

## LDA (usando Count)

In [61]:
#all_datasets
lda = LatentDirichletAllocation(n_components=70, max_iter=50, random_state=0, n_jobs=5)
normalizer = MinMaxScaler(copy=False)
lda = make_pipeline(lda, normalizer)
X_lda = lda.fit_transform(X_count)

print("\nall_datasets")
for classifier in classifiers:
    try:
        acc = run_ml_model(classifier, **split_data(X_lda, avalencias))
        matriz_resultados['all']['count+lda'][classifier.__class__.__name__] = acc
    except:
        pass


#tweets_mg
X_lda = lda.fit_transform(X_count_tmg)

print("\ntweets_mg")
for classifier in classifiers:
    try:
        acc = run_ml_model(classifier, **split_data(X_lda, aval_tweets_mg))
        matriz_resultados['tweets']['count+lda'][classifier.__class__.__name__] = acc
    except:
        pass
      
      
#titulo_noticias
X_lda = lda.fit_transform(X_count_tn)

print("\ntitulo_noticias")
for classifier in classifiers:
    try:
        acc = run_ml_model(classifier, **split_data(X_lda, aval_titulo_noticias))
        matriz_resultados['titulo_noticias']['count+lda'][classifier.__class__.__name__] = acc
    except:
        pass


all_datasets
Modelo   : MultinomialNB
Acurácia : 50.92%
--------------------
Modelo   : ComplementNB
Acurácia : 48.67%
--------------------
Modelo   : LogisticRegression
Acurácia : 50.5%
--------------------
Modelo   : RandomForestClassifier
Acurácia : 52.93%
--------------------
Modelo   : KNeighborsClassifier
Acurácia : 45.53%
--------------------
Modelo   : MLPClassifier
Acurácia : 47.6%
--------------------
Modelo   : LinearSVC
Acurácia : 50.15%
--------------------
Modelo   : SVC
Acurácia : 47.07%
--------------------

tweets_mg
Modelo   : MultinomialNB
Acurácia : 55.01%
--------------------
Modelo   : ComplementNB
Acurácia : 55.21%
--------------------
Modelo   : LogisticRegression
Acurácia : 56.83%
--------------------
Modelo   : RandomForestClassifier
Acurácia : 56.02%
--------------------
Modelo   : KNeighborsClassifier
Acurácia : 49.44%
--------------------
Modelo   : MLPClassifier
Acurácia : 55.01%
--------------------
Modelo   : LinearSVC
Acurácia : 56.42%
----------------

## Count + TF-IDF + Word2Vec

In [62]:
#all_datasets
# Count
vec_count = CountVectorizer()
X_count = vec_count.fit_transform(afrases)
weights_count = pd.DataFrame(np.round(X_count.toarray().T, 8), index=vec_count.get_feature_names())

# TF-IDF
vec_tfidf = TfidfVectorizer()
X_tfidf = vec_tfidf.fit_transform(afrases)
weights_tfidf = pd.DataFrame(np.round(X_tfidf.toarray().T, 8), index=vec_tfidf.get_feature_names())

# Word2Vec preprocessing
frases_w2v = []
for frase in afrases:
    bigram = []
    p_frase = word_tokenize(frase)
    for m, palavra in enumerate(p_frase):
        next_p = None
        try:
            next_p = p_frase[m+1]
        except:
            pass
        bigram += [f'{palavra}']
#         if next_p:
#             bigram += [f'{palavra} {next_p}']
    frases_w2v += [bigram]

# Word2Vec
model = gensim.models.Word2Vec(
    sentences=frases_w2v,
    sg=1,
    hs=1,
    size=1,
    window=25,
    min_count=1,
    seed=0,
    workers=10)
model.train(frases_w2v, total_examples=len(frases_w2v), epochs=1000)

(37612056, 43013000)

In [63]:
#all_datasets
r_words = {}
for word in vec_count.get_feature_names():
    idx = weights_count.index.get_loc(word)
    w2c_val = .1
    try:
        w2c_val = model.wv[word]
    except:
        pass
    r_words[word] = (weights_tfidf.iloc[idx].values + weights_count.iloc[idx].values) * w2c_val
lwor = list(r_words.keys())
X = np.asarray(list(r_words.values()))
weights = pd.DataFrame(X, index=lwor)
X = X.T

normalizer = Normalizer(copy=False)
X = normalizer.fit_transform(X)

print("\nall_datasets")
for classifier in classifiers:
    try:
        acc = run_ml_model(classifier, **split_data(X, avalencias))
        matriz_resultados['all']['tfidf+count+w2c'][classifier.__class__.__name__] = acc
    except:
        pass


all_datasets
Modelo   : LogisticRegression
Acurácia : 66.25%
--------------------
Modelo   : RandomForestClassifier
Acurácia : 65.78%
--------------------
Modelo   : KNeighborsClassifier
Acurácia : 61.04%
--------------------
Modelo   : MLPClassifier
Acurácia : 62.29%
--------------------
Modelo   : LinearSVC
Acurácia : 65.6%
--------------------
Modelo   : SVC
Acurácia : 57.37%
--------------------


In [64]:
#tweets_mg
# Count
vec_count = CountVectorizer()
X_count = vec_count.fit_transform(atweets_mg)
weights_count = pd.DataFrame(np.round(X_count.toarray().T, 8), index=vec_count.get_feature_names())

# TF-IDF
vec_tfidf = TfidfVectorizer()
X_tfidf = vec_tfidf.fit_transform(atweets_mg)
weights_tfidf = pd.DataFrame(np.round(X_tfidf.toarray().T, 8), index=vec_tfidf.get_feature_names())

# Word2Vec preprocessing
frases_w2v = []
for frase in atweets_mg:
    bigram = []
    p_frase = word_tokenize(frase)
    for m, palavra in enumerate(p_frase):
        next_p = None
        try:
            next_p = p_frase[m+1]
        except:
            pass
        bigram += [f'{palavra}']
#         if next_p:
#             bigram += [f'{palavra} {next_p}']
    frases_w2v += [bigram]

# Word2Vec
model = gensim.models.Word2Vec(
    sentences=frases_w2v,
    sg=1,
    hs=1,
    size=1,
    window=25,
    min_count=1,
    seed=0,
    workers=10)
model.train(frases_w2v, total_examples=len(frases_w2v), epochs=1000)

(21300882, 26219000)

In [65]:
#tweets_mg
r_words = {}
for word in vec_count.get_feature_names():
    idx = weights_count.index.get_loc(word)
    w2c_val = .1
    try:
        w2c_val = model.wv[word]
    except:
        pass
    r_words[word] = (weights_tfidf.iloc[idx].values + weights_count.iloc[idx].values) * w2c_val
lwor = list(r_words.keys())
X = np.asarray(list(r_words.values()))
weights = pd.DataFrame(X, index=lwor)
X = X.T

normalizer = Normalizer(copy=False)
X = normalizer.fit_transform(X)

print("\ntweets_mg")
for classifier in classifiers:
    try:
        acc = run_ml_model(classifier, **split_data(X, aval_tweets_mg))
        matriz_resultados['tweets']['tfidf+count+w2c'][classifier.__class__.__name__] = acc
    except:
        pass


tweets_mg
Modelo   : LogisticRegression
Acurácia : 64.71%
--------------------
Modelo   : RandomForestClassifier
Acurácia : 64.81%
--------------------
Modelo   : KNeighborsClassifier
Acurácia : 61.78%
--------------------
Modelo   : MLPClassifier
Acurácia : 58.65%
--------------------
Modelo   : LinearSVC
Acurácia : 62.29%
--------------------
Modelo   : SVC
Acurácia : 62.49%
--------------------


In [66]:
#titulo_noticias
# Count
vec_count = CountVectorizer()
X_count = vec_count.fit_transform(atitulo_noticias)
weights_count = pd.DataFrame(np.round(X_count.toarray().T, 8), index=vec_count.get_feature_names())

# TF-IDF
vec_tfidf = TfidfVectorizer()
X_tfidf = vec_tfidf.fit_transform(atitulo_noticias)
weights_tfidf = pd.DataFrame(np.round(X_tfidf.toarray().T, 8), index=vec_tfidf.get_feature_names())

# Word2Vec preprocessing
frases_w2v = []
for frase in atitulo_noticias:
    bigram = []
    p_frase = word_tokenize(frase)
    for m, palavra in enumerate(p_frase):
        next_p = None
        try:
            next_p = p_frase[m+1]
        except:
            pass
        bigram += [f'{palavra}']
#         if next_p:
#             bigram += [f'{palavra} {next_p}']
    frases_w2v += [bigram]

# Word2Vec
model = gensim.models.Word2Vec(
    sentences=frases_w2v,
    sg=1,
    hs=1,
    size=1,
    window=25,
    min_count=1,
    seed=0,
    workers=10)
model.train(frases_w2v, total_examples=len(frases_w2v), epochs=1000)

(13989419, 16794000)

In [67]:
#titulo_noticias
r_words = {}
for word in vec_count.get_feature_names():
    idx = weights_count.index.get_loc(word)
    w2c_val = .1
    try:
        w2c_val = model.wjv[word]
    except:
        pass
    r_words[word] = (weights_tfidf.iloc[idx].values + weights_count.iloc[idx].values) * w2c_val
lwor = list(r_words.keys())
X = np.asarray(list(r_words.values()))
weights = pd.DataFrame(X, index=lwor)
X = X.T

normalizer = Normalizer(copy=False)
X = normalizer.fit_transform(X)

print("\ntitulo_noticias")
for classifier in classifiers:
    try:
        acc = run_ml_model(classifier, **split_data(X, aval_titulo_noticias))
        matriz_resultados['titulo_noticias']['tfidf+count+w2c'][classifier.__class__.__name__] = acc
    except:
        pass


titulo_noticias
Modelo   : MultinomialNB
Acurácia : 60.91%
--------------------
Modelo   : ComplementNB
Acurácia : 65.19%
--------------------
Modelo   : LogisticRegression
Acurácia : 66.19%
--------------------
Modelo   : RandomForestClassifier
Acurácia : 63.34%
--------------------
Modelo   : KNeighborsClassifier
Acurácia : 57.92%
--------------------
Modelo   : MLPClassifier
Acurácia : 63.2%
--------------------
Modelo   : LinearSVC
Acurácia : 66.76%
--------------------
Modelo   : SVC
Acurácia : 47.93%
--------------------


### Resultado dos Classificadores

#### Resultado dos Classificadores para todas as frases

In [68]:
df = pd.DataFrame.from_dict(matriz_resultados['all'])
df.style.apply(highlight_max).format({
    'tfidf': '{:,.2f}%'.format,
    'tfidf+lsa': '{:,.2f}%'.format,
    'tfidf+lda': '{:,.2f}%'.format,
    'count': '{:,.2f}%'.format,
    'count+lsa': '{:,.2f}%'.format,
    'count+lda': '{:,.2f}%'.format,
    'tfidf+count+w2c': '{:,.2f}%'.format,
})

,tfidf,tfidf+lsa,tfidf+lda,count,count+lsa,count+lda,tfidf+count+w2c
ComplementNB,62.17%,53.52%,43.04%,59.50%,54.17%,48.67%,0.00%
KNeighborsClassifier,61.16%,57.90%,43.10%,40.20%,57.79%,45.53%,61.04%
LinearSVC,67.32%,62.52%,46.54%,65.01%,61.93%,50.15%,65.60%
LogisticRegression,66.43%,61.99%,46.36%,65.96%,61.10%,50.50%,66.25%
MLPClassifier,64.18%,62.70%,47.13%,63.11%,61.81%,47.60%,62.29%
MultinomialNB,61.63%,45.41%,46.65%,65.19%,46.48%,50.92%,0.00%
RandomForestClassifier,63.88%,61.28%,45.71%,65.48%,59.86%,52.93%,65.78%
SVC,58.26%,47.72%,46.77%,49.44%,46.71%,47.07%,57.37%


#### Resultado dos Classificadores para os Tweets MG

In [69]:
df = pd.DataFrame.from_dict(matriz_resultados['tweets'])
df.style.apply(highlight_max).format({
    'tfidf': '{:,.2f}%'.format,
    'tfidf+lsa': '{:,.2f}%'.format,
    'tfidf+lda': '{:,.2f}%'.format,
    'count': '{:,.2f}%'.format,
    'count+lsa': '{:,.2f}%'.format,
    'count+lda': '{:,.2f}%'.format,
    'tfidf+count+w2c': '{:,.2f}%'.format,
})

,tfidf,tfidf+lsa,tfidf+lda,count,count+lsa,count+lda,tfidf+count+w2c
ComplementNB,59.35%,56.62%,48.33%,59.66%,58.65%,55.21%,0.00%
KNeighborsClassifier,60.67%,60.87%,46.51%,51.37%,61.68%,49.44%,61.78%
LinearSVC,62.49%,65.42%,53.29%,60.57%,65.22%,56.42%,62.29%
LogisticRegression,65.62%,66.03%,53.29%,61.07%,64.41%,56.83%,64.71%
MLPClassifier,60.57%,66.53%,51.47%,60.47%,66.13%,55.01%,58.65%
MultinomialNB,63.50%,44.29%,52.58%,60.57%,50.46%,55.01%,0.00%
RandomForestClassifier,64.91%,62.08%,50.96%,63.80%,62.39%,56.02%,64.81%
SVC,60.97%,45.70%,46.51%,44.69%,45.10%,47.62%,62.49%


#### Resultados dos Classificadores para os Títulos de Notícias

In [70]:
df = pd.DataFrame.from_dict(matriz_resultados['titulo_noticias'])
df.style.apply(highlight_max).format({
    'tfidf': '{:,.2f}%'.format,
    'tfidf+lsa': '{:,.2f}%'.format,
    'tfidf+lda': '{:,.2f}%'.format,
    'count': '{:,.2f}%'.format,
    'count+lsa': '{:,.2f}%'.format,
    'count+lda': '{:,.2f}%'.format,
    'tfidf+count+w2c': '{:,.2f}%'.format,
})

,tfidf,tfidf+lsa,tfidf+lda,count,count+lsa,count+lda,tfidf+count+w2c
ComplementNB,65.05%,58.06%,37.38%,62.77%,52.78%,41.80%,65.19%
KNeighborsClassifier,59.06%,56.06%,38.23%,35.66%,53.64%,39.94%,57.92%
LinearSVC,66.33%,61.48%,41.08%,65.05%,60.49%,45.51%,66.76%
LogisticRegression,66.19%,60.34%,41.23%,66.19%,60.49%,45.65%,66.19%
MLPClassifier,65.76%,58.63%,40.80%,65.19%,55.63%,43.94%,63.20%
MultinomialNB,60.91%,42.51%,42.80%,68.33%,42.51%,44.22%,60.91%
RandomForestClassifier,62.48%,62.62%,46.08%,63.34%,58.77%,46.08%,63.34%
SVC,48.36%,43.79%,43.37%,44.22%,43.94%,43.22%,47.93%
